In [2]:
import sys
sys.path.append('../')

from final_utils import *

gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

from ResNet_FC import *
from ResNet import *
import time

2022-12-06 14:31:31.006409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 14:31:31.006878: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 14:31:31.010891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 14:31:31.011326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 14:31:31.011943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [3]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.set_visible_devices(gpus[1], 'GPU')
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)

2 Physical GPUs, 1 Logical GPU


2022-12-06 14:31:35.765711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 14:31:35.767038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 14:31:35.767870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 14:31:35.768615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
root_path = generate_root_path()
SC_CE_KLD = tf.keras.losses.SparseCategoricalCrossentropy()

### DataLoader and other nesseccary functions

In [5]:
def binary_accuracy(y_true, y_pred): 
    return ((tf.math.argmax(y_pred, axis=1)) == tf.cast(y_true, dtype=tf.dtypes.int64)).numpy().mean()

def lr_exp_decay(epoch, lr):
    k = 0.04
    return lr * np.exp(-k*epoch)


def load_batch(paths):
    images = []
    labels = []
    for i,path in enumerate(paths):
        try:
            img = cv2.imread(path, cv2.COLOR_BGR2RGB)
            img = img.astype('float32')
            img /= 255 
            if 'good' in path:
                label = 1
            elif 'bad' in path:
                label = 0
        except:
            continue
            
        images.append(img)
        labels.append(label)
    
    images = np.stack(images)

    
    return images, labels

### Trainer

In [6]:
def trainer(model, data, data_val,batch_size=128, epochs=50, learning_rate=0.03, verbose=0):
    X_train, y_train = load_batch(data)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)#.prefetch(tf.data.AUTOTUNE)  # ToDo: here use .cache()

    history = model.fit(train_dataset,
                        batch_size=batch_size,
                        epochs=1,
                        verbose=verbose,
                        validation_data = data_val)

    return history

### Loading data

In [9]:
# main_path = main_path=f'{root_path}Data/AesthAI/alm/splitted/alm_train/'
# paths_bad = []
# paths_good = []
    
# for i in range(7):
#     alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
#     bad_data = json.load(alm_train_bad)
    
#     for data in bad_data:
#         path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
#         paths_bad.append(path_to_img)
        
# alm_train_good = open(f'{main_path}/data_good1.json')
# good_data = json.load(alm_train_good)
# for data in good_data:
#     path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
#     paths_good.append(path_to_img)
    
# for i in range(7):
#     paths_bad[i] = np.squeeze(np.array(paths_bad[i]))
# paths_good = np.squeeze(np.array(paths_good))
   
# # Generating static validation data
# paths_bad, paths_bad_val = extract_static_val_data(paths_bad, perc = 0.017) #original - 0.017
# paths_good, paths_good_val = extract_static_val_data(paths_good, perc = 0.11) #original - 0.11

# paths_bad = np.array(paths_bad)
# paths_good = np.array(paths_good)
# paths_bad_val = np.array(paths_bad_val)
# paths_good_val = np.array(paths_good_val)
# labels_good = np.ones(len(paths_good))
# labels_bad = np.zeros(len(paths_bad))

main_path = main_path=f'{root_path}Data/AesthAI/alm/splitted/alm_train/'
paths_bad = []
paths_good = []
    
for i in range(7):
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    
    for data in bad_data:
        path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
        paths_bad.append(path_to_img)
        
alm_train_good = open(f'{main_path}/data_good1.json')
good_data = json.load(alm_train_good)
for data in good_data:
    path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
    paths_good.append(path_to_img)
    
for i in range(7):
    paths_bad[i] = np.squeeze(np.array(paths_bad[i]))
paths_good = np.squeeze(np.array(paths_good))
   
# Generating static validation data
paths_bad, paths_bad_val = extract_static_val_data(paths_bad, perc = 0.014) #original - 0.017
paths_good, paths_good_val = extract_static_val_data(paths_good, perc = 0.04) #original - 0.11

paths_bad = np.array(paths_bad)
paths_bad_val = np.array(paths_bad_val)
paths_good = np.array(paths_good)
paths_good_val = np.array(paths_good_val)

full_data = np.concatenate((np.repeat(paths_good, 7), paths_bad))
    
#shuffling
idx = np.random.permutation(len(full_data))
full_data = full_data[idx]
# full_data = full_data[:5000]  # debug
paths_good_val.shape

(470,)

In [ ]:
# full_data = np.concatenate((np.repeat(paths_good, 7), paths_bad))
# full_labels = np.concatenate(((np.repeat(labels_good, 7), labels_bad)))   

# #shuffling
# idx = np.random.permutation(len(full_data))
# full_data = full_data[idx]
# print(full_data.shape)
# print(full_labels.shape)

In [10]:
# #Splitting data 
# split_factor = 1024
# splitted_data = []
# splitted_labels = []

# global_batches = int(full_data.shape[0] / split_factor)
# for i in range(global_batches):
#     batch_data = full_data[i*split_factor: (i+1)*split_factor]
#     batch_labels = full_labels[i*split_factor: (i+1)*split_factor]
    
#     splitted_labels.append(batch_labels)
#     splitted_data.append(batch_data)
    

# splitted_data[-1] = np.concatenate((splitted_data[-1], full_data[len(splitted_data)*split_factor:]))
# splitted_labels[-1] = np.concatenate((splitted_labels[-1], full_labels[len(splitted_labels)*split_factor:]))

# data = splitted_data, splitted_labels

split_factor = 1024
splitted_data = []

global_batches = int(full_data.shape[0] / split_factor)
for i in range(global_batches):
    batch_data = full_data[i*split_factor: (i+1)*split_factor]
    splitted_data.append(batch_data)
    
splitted_data.append(full_data[len(splitted_data)*split_factor:])

data = splitted_data

In [11]:
#Loading validation data
paths_val = np.concatenate((paths_bad_val, paths_good_val ), axis=0 )
X_val, y_val = load_batch(paths_val)
data_val = (X_val, y_val)
data_val = tf.data.Dataset.from_tensor_slices(data_val).batch(32)#.prefetch(tf.data.AUTOTUNE)   # ToDo: also use .prefetch(tf.data.AUTOTUNE)

2022-12-06 14:33:06.538561: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6302880000 exceeds 10% of free system memory.
2022-12-06 14:33:11.736379: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6302880000 exceeds 10% of free system memory.


### Creating model and training

In [16]:
batch_size = 32
epochs = 50
learning_rate = 0.003

In [19]:
# model for resnet cnn fc training
model_resnet = ResNet18(num_classes=5000)
model_resnet.build((None,600,600,3))
resnet_weights_path = '../models/ResNet/ResNet_original_border_600x600_best_94.59.hdf5'
model_resnet.load_weights(resnet_weights_path)

model_fc =  model = fc_model_softmax(input_num=5000)
fc_weights_path = '../models/Softmax/FC_For_Resnet/best_custom_94_59_train.hdf5'
model_fc.load_weights(fc_weights_path)

model = resnet_fc(model_resnet, model_fc)
model.build((None,600,600,3))
weights_path = '../models/Softmax/ResNet_FC/resnet_fc_joint_5k_94_59_test.hdf5'
model.save_weights(weights_path)
model.load_weights(weights_path)

In [20]:
model.summary()

Model: "resnet_fc_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_net18_1 (ResNet18)      multiple                  4074504   
                                                                 
 model_1 (Functional)        (None, 2)                 12616450  
                                                                 
Total params: 16,690,954
Trainable params: 16,679,818
Non-trainable params: 11,136
_________________________________________________________________


In [23]:
run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

epochs = 1

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}: ')
    learning_rate = lr_exp_decay(epoch, learning_rate)
    print('Learnin rate:', learning_rate)
    model.compile(loss=SC_CE_KLD,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                                  epsilon=1e-07, decay=0, amsgrad=False), options = run_opts)
    if epoch != 0:
        model.load_weights(weights_path)
        
    random.shuffle(data)
        
    for i in range(len(data)):
        verbose = 1
        if i % 10 == 0:
            print(i)
            verbose = 1
            model.save_weights(weights_path)
            
        
        batch_data = np.array(data[i])
        random.shuffle(batch_data)

#         k = time.time()
        history = trainer(model, 
                          batch_data, 
                          data_val,
                          batch_size=batch_size,
                          epochs=epochs,
                          learning_rate=learning_rate,
                          verbose=verbose)    
#         print('1 mini batch training: ', time.time() - k)

#     model.save_weights(f'models/shufflenet_fc/Shufflenet_fc_21.09_512.h5', save_format='h5')
    print('Done, epoch training!')

Epoch 1/1: 
Learnin rate: 0.003


TypeError: Invalid keyword argument(s) in `compile()`: ({'options'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

### Training on dummy data

In [ ]:
img_data = tf.random.uniform((64, 600, 600, 3))
labels = np.random.randint(0,1,(64, 16928))
data = img_data, labels

In [ ]:
img_data = tf.random.uniform((4, 600, 600, 3))
labels = np.random.randint(0,1,(4, 16928))
data_val = img_data, labels

In [ ]:
type(labels)

In [ ]:
labels = tf.convert_to_tensor(labels)

In [ ]:
type(img_data)

In [ ]:
history = trainer(model, 
                  data, 
                  weights_path,
                  data_val,
                  batch_size=batch_size,
                  epochs=epochs, 
                  learning_rate=learning_rate)